In [2]:
from langchain_community.document_loaders import GitLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
from langchain_core.documents import Document
from dotenv import load_dotenv

/Users/chaitanyaneware/Developer/gen-ai_Pro/get_insight/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

load_dotenv()
output_parser = StrOutputParser()

In [ ]:
# 1️⃣ Load repo
loader = GitLoader(
    clone_url="https://github.com/chaitanyaSn/Fintech-Bank-Wallet.git",
    repo_path="./temp_clone",
    branch="main"
)


In [12]:
docs = loader.load()
print(f"\n📚 Total documents loaded: {len(docs)}")


📚 Total documents loaded: 119


In [13]:
# 2️⃣ Split into code-aware chunks
splitter = RecursiveCharacterTextSplitter.from_language(
    language="java",
    chunk_size=2000,
    chunk_overlap=400
)
chunks = splitter.split_documents(docs)
print(f"\n📄 Total chunks created: {len(chunks)}")
print(chunks[:4])  # Print first 2 chunks for inspection


📄 Total chunks created: 212
[Document(metadata={'source': 'README.md', 'file_path': 'README.md', 'file_name': 'README.md', 'file_type': '.md'}, page_content='# 💰 Digital Wallet Microservices Platform\n\n![Architecture](https://github.com/user-attachments/assets/05d876f2-0d59-47fc-9cfa-bc12b059367b)\n\nA scalable microservices-based digital wallet platform with secure transactions and real-time notifications.\n\n\n## 🏗️ Architecture\n\n**🌐 Client Layer:** Postman/Web Apps → REST API requests  \n**🚪 Gateway:** API Gateway (JWT auth, rate limiting, load balancing)  \n**🎯 Services:** User, Wallet, Transaction, Notification microservices  \n**🔍 Infrastructure:** Eureka (discovery), Redis (cache), Kafka (messaging), Docker  \n\n## 🔄 Flow\n```\nClient → API Gateway → Auth Filter → Service Discovery → Target Service\nUser Sign Up → Wallet Creation → Transaction Processing → Kafka Event → Notification\n```\n\n\n## 🎯 Services\n\n| Service | Port | Purpose |\n|---------|------|---------|\n| 🚪 **

In [16]:
# 3️⃣ Create embeddings with Hugging Face model
print("🔄 Initializing embeddings...")
embeddings = HuggingFaceEmbeddings(model="nomic-ai/nomic-embed-text-v1", model_kwargs={"trust_remote_code": True})

🔄 Initializing embeddings...


<All keys matched successfully>


In [18]:
# 4️⃣ Create and persist FAISS index
print("💾 Creating vector store...")
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_index")
print("✅ Vector store created and saved locally.")

💾 Creating vector store...
✅ Vector store created and saved locally.


In [19]:
# 5️⃣ Build retriever + LLM
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [20]:
# 6️⃣ Prompt Template
prompt = PromptTemplate(
    template="""You are a helpful assistant analyzing a GitHub repository.
Answer the question based only on the context provided below.
If you don't know, just say you don't know.

Context:
{context}

Question:
{question}

Answer:""",
    input_variables=["context", "question"]
)

In [21]:
# 7️⃣ Ask Question
question = "how rate limiter works int his project?"
print(f"\n❓ Question: {question}")


❓ Question: how rate limiter works int his project?


In [23]:
retrieved_docs = retriever.invoke(question)
context_text = " ".join([doc.page_content for doc in retrieved_docs])


In [24]:
final_prompt = prompt.format(context=context_text, question=question)
response = llm.invoke(final_prompt)
final_res = output_parser.invoke(response)  
print("\n🤖 Response:\n", final_res)


🤖 Response:
 In this project, rate limiting is implemented through the Spring Cloud Gateway and specifically applied to the `TransactionMs` service route.

Here's how it works:

1.  **Key Resolution**: The `ipKeyResolver` bean in `RateLimiterConfig` is used to identify the client making the request. It extracts the client's IP address from `exchange.getRequest().getRemoteAddress().getAddress().getHostAddress()` and uses it as the key for rate limiting. This means each unique IP address will be rate-limited independently.

2.  **Gateway Filter**: A `RequestRateLimiter` filter is configured for the `TransactionMs` route in the API Gateway. This filter uses a Redis-backed rate limiter.

3.  **Rate Limiting Parameters**:
    *   **Replenish Rate**: `redis-rate-limiter.replenishRate=5`. This means the rate limiter allows 5 requests per unit of time (typically per second, depending on the RedisRateLimiter implementation).
    *   **Burst Capacity**: `redis-rate-limiter.burstCapacity=10`. Th